In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import make_column_transformer
import copy
from pandas import Series
import netaddr, time, sys, json, zipfile, csv, geopandas, re, os
import pandas as pd
from zipfile import ZipFile
from io import TextIOWrapper, BytesIO
from collections import defaultdict
from matplotlib import pyplot as plt

In [2]:
train_users = pd.read_csv("data/train_users.csv")
train_logs = pd.read_csv("data/train_logs.csv")
train_y = pd.read_csv("data/train_y.csv")
test_users = pd.read_csv("data/test1_users.csv")
test_logs = pd.read_csv("data/test1_logs.csv")
test_y = pd.read_csv("data/test1_y.csv")
ips = pd.read_csv("data/ip2location.csv")

In [3]:
def total_min_ips(logs, users):    
    minutes = {}
    avg_min = {}
    num_visited = {}
    netaddrs = {}
    ids = set(users["id"])
    for id_ in ids:
        minutes[id_] = logs[logs["id"] == id_]["minutes_on_page"].sum()
        avg_min[id_] = logs[logs["id"] == id_]["minutes_on_page"].mean()
        num_visited[id_] = len(logs[logs["id"] == id_])
        try:
            netaddrs[id_] = int(netaddr.IPAddress(logs[logs["id"] == id_]["ip_address"].values[-1]))
        except:
            netaddrs[id_] = 0
    users["total_min"] = minutes.values()
    users["avg_minutes"] = avg_min.values()
    users["num_visited"] = num_visited.values()
    users["netaddr"] = netaddrs.values()
    return users.fillna(0)

In [4]:
train_users = total_min_ips(train_logs, train_users)
test_users = total_min_ips(test_logs, test_users)

In [5]:
def ip_to_country(df):
    df = df.sort_values("netaddr")
    countries = list()
    ip_idx = 0
    for ip in df["netaddr"]:
        for low in ips["low"][ip_idx:]:
            if low <= ip <= ips["high"][ip_idx]:
                countries.append(ips["region"][ip_idx])
                break
            else:
                ip_idx += 1
    df["country"] = countries
    return df.sort_values("id")

In [6]:
train_users = ip_to_country(train_users)
test_users = ip_to_country(test_users)
merged_train = pd.merge(left=train_users, right=train_y, left_on="id", right_on="id")
merged_test = pd.merge(left=test_users, right=test_y, left_on="id", right_on="id")

In [7]:
merged_train

,id,name,age,badge,past_purchase_amt,total_min,avg_minutes,num_visited,netaddr,country,y
0,1,Nikki Young,36,bronze,42.94,58.288339,11.657668,5,1744111453,Japan,1
1,2,William Moats,40,gold,10.03,83.692930,10.461616,8,2110788436,China,0
2,3,John Lemke,24,silver,203.37,70.934103,7.881567,9,3406882002,China,0
3,4,Elizabeth Gavin,39,bronze,132.04,60.024907,7.503113,8,1951432363,Japan,0
4,5,Myrtle Blais,37,bronze,41.56,71.578302,7.953145,9,2709193103,Japan,0
...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,Daniel Ortiz,34,gold,37.54,140.712992,7.405947,19,3389899756,Japan,0
19996,19997,Donna Mccarty,38,bronze,12.89,94.782392,9.478239,10,280942397,United States of America,0
19997,19998,Hortensia Frazier,34,silver,22.52,49.576218,9.915244,5,767197018,Mexico,0
19998,19999,Harold Eichhorn,40,bronze,14.02,111.350570,10.122779,11,675820811,China,0


In [8]:
merged_test

,id,name,age,badge,past_purchase_amt,total_min,avg_minutes,num_visited,netaddr,country,y
0,100001,William Lee,33,bronze,10.12,55.198310,9.199718,6,2500175163,United States of America,0
1,100002,Ernest Glover,37,bronze,21.22,58.283964,8.326281,7,3450272726,Mexico,0
2,100003,James Thompson,31,silver,18.02,61.078669,8.725524,7,399379405,Japan,1
3,100004,Lillie Yates,33,gold,43.41,79.029091,7.902909,10,3394515195,Japan,1
4,100005,George Schaeffer,32,bronze,11.55,102.252520,8.521043,12,2392016507,United States of America,0
...,...,...,...,...,...,...,...,...,...,...,...
19995,119996,Steven Grosser,26,silver,95.34,130.837112,11.894283,11,762410000,Japan,1
19996,119997,Sylvie League,44,silver,21.40,92.952199,8.450200,11,386888437,Mexico,1
19997,119998,Eula Cleveland,36,bronze,34.26,105.954793,8.829566,12,3628382185,United States of America,0
19998,119999,Lisa Couture,27,silver,11.84,54.145469,9.024245,6,3007171586,United States of America,1


In [9]:
train_logs

,date,id,ip_address,url_visited,minutes_on_page
0,6-14-2020,19083,123.196.90.97,/blender.html,7.222454
1,8-22-2020,764,123.50.162.196,/cleats.html,5.472573
2,9-14-2020,733,192.141.247.60,/tablet.html,8.396012
3,8-27-2020,16282,203.28.112.62,/keyboard.html,4.139941
4,7-29-2020,5694,23.78.141.31,/cleats.html,11.204377
...,...,...,...,...,...
199995,7-14-2020,10299,108.61.207.238,/cooler.html,17.208746
199996,4-17-2020,981,45.124.21.4,/basketball.html,10.926431
199997,9-27-2020,10451,103.15.97.131,/monitor.html,11.502494
199998,4-13-2020,2885,203.26.56.100,/keyboard.html,5.056544


In [10]:
test_logs

,date,id,ip_address,url_visited,minutes_on_page
0,8-7-2020,111456,168.197.43.0,/monitor.html,3.735996
1,7-30-2020,115915,125.252.64.22,/cleats.html,10.796966
2,3-31-2020,111309,138.186.29.158,/cooler.html,5.811202
3,11-22-2020,113821,16.251.44.238,/cooler.html,12.640072
4,2-29-2020,105981,202.142.28.59,/laptop.html,2.495160
...,...,...,...,...,...
199995,2-7-2020,100832,103.99.234.174,/bicycle.html,11.049550
199996,10-1-2020,102602,169.46.152.141,/spatula.html,7.307361
199997,1-24-2020,102018,203.55.218.123,/lamp.html,7.490139
199998,9-8-2020,102914,203.191.76.154,/basketball.html,5.270011


In [11]:
merged_train[merged_train["total_min"] == 0]

,id,name,age,badge,past_purchase_amt,total_min,avg_minutes,num_visited,netaddr,country,y
5099,5100,Patricia Tucker,46,bronze,11.84,0.0,0.0,0,0,-,1
7835,7836,Christina Tuten,37,silver,20.15,0.0,0.0,0,0,-,1


In [94]:
pipeline = Pipeline([
    ("both", make_column_transformer((OneHotEncoder(), ["badge", "age", "country"]), 
                                     (PolynomialFeatures(degree=1, include_bias=False), ["past_purchase_amt", "total_min", "avg_minutes", "num_visited"]), remainder="passthrough")),
    ("lr", LogisticRegression(solver="lbfgs", max_iter=1025))
])
pipeline.fit(merged_train[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]], merged_train["y"])



Pipeline(steps=[('both',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['badge', 'age', 'country']),
                                                 ('polynomialfeatures',
                                                  PolynomialFeatures(degree=1,
                                                                     include_bias=False),
                                                  ['past_purchase_amt',
                                                   'total_min', 'avg_minutes',
                                                   'num_visited'])])),
                ('lr', LogisticRegression(max_iter=1025))])

In [95]:
pipeline.score(merged_test[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]], merged_test["y"])



0.7907

In [57]:
merged_train["predicted"] = pipeline.predict(merged_train[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]])
merged_test["predicted"] = pipeline.predict(merged_test[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]])

In [58]:
merged_train

,id,name,age,badge,past_purchase_amt,total_min,avg_minutes,num_visited,netaddr,country,y,predicted
0,1,Nikki Young,36,bronze,42.94,58.288339,11.657668,5,1744111453,Japan,1,0
1,2,William Moats,40,gold,10.03,83.692930,10.461616,8,2110788436,China,0,0
2,3,John Lemke,24,silver,203.37,70.934103,7.881567,9,3406882002,China,0,0
3,4,Elizabeth Gavin,39,bronze,132.04,60.024907,7.503113,8,1951432363,Japan,0,0
4,5,Myrtle Blais,37,bronze,41.56,71.578302,7.953145,9,2709193103,Japan,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,Daniel Ortiz,34,gold,37.54,140.712992,7.405947,19,3389899756,Japan,0,0
19996,19997,Donna Mccarty,38,bronze,12.89,94.782392,9.478239,10,280942397,United States of America,0,0
19997,19998,Hortensia Frazier,34,silver,22.52,49.576218,9.915244,5,767197018,Mexico,0,1
19998,19999,Harold Eichhorn,40,bronze,14.02,111.350570,10.122779,11,675820811,China,0,0


In [59]:
merged_test

,id,name,age,badge,past_purchase_amt,total_min,avg_minutes,num_visited,netaddr,country,y,predicted
0,100001,William Lee,33,bronze,10.12,55.198310,9.199718,6,2500175163,United States of America,0,0
1,100002,Ernest Glover,37,bronze,21.22,58.283964,8.326281,7,3450272726,Mexico,0,0
2,100003,James Thompson,31,silver,18.02,61.078669,8.725524,7,399379405,Japan,1,1
3,100004,Lillie Yates,33,gold,43.41,79.029091,7.902909,10,3394515195,Japan,1,0
4,100005,George Schaeffer,32,bronze,11.55,102.252520,8.521043,12,2392016507,United States of America,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,119996,Steven Grosser,26,silver,95.34,130.837112,11.894283,11,762410000,Japan,1,1
19996,119997,Sylvie League,44,silver,21.40,92.952199,8.450200,11,386888437,Mexico,1,1
19997,119998,Eula Cleveland,36,bronze,34.26,105.954793,8.829566,12,3628382185,United States of America,0,0
19998,119999,Lisa Couture,27,silver,11.84,54.145469,9.024245,6,3007171586,United States of America,1,1


In [60]:
merged_test["predicted"].values

array([0, 0, 1, ..., 0, 1, 1])